```python
## \file /sandbox/davidka/crawler.py
# -*- coding: utf-8 -*-
#! .pyenv/bin/python3
```

Модуль для сбора данных со страниц различных сайтов
=====================================================


```rst
.. module:: sandbox.davidka.crawler
```


In [ ]:
#%pip install ipdb


In [1]:
import ipdb # <- трасировка и точки останова
import asyncio, random
from pathlib import Path
from types import SimpleNamespace


import header
from header import __root__
from src import gs
from src.webdriver.ai_browser.simple_browser import SimpleDriver
from src.utils.jjson import j_loads, j_loads_ns, j_dumps
from src.utils.file import read_text_file, save_text_file, get_filenames_from_directory 
from src.utils.printer import pprint as print
from src.logger.logger import logger

2025-04-23 23:30:28,866 - ERROR - ❌ Error fetching data: 404 
NoneType: None
2025-04-23 23:30:32,970 - INFO - Anonymized telemetry enabled. See https://docs.browser-use.com/development/telemetry for more information.
2025-04-23 23:30:32,975 - WARNING - ⚠️ Не удалось импортировать BrowserController. Браузерные инструменты будут недоступны. No module named 'src.webdriver.ai_browser.controlers'
2025-04-23 23:30:32,976 - INFO - ℹ️ Config Gemini: Status=active, Model=gemini-1.5-flash-latest, Key Present=True 
2025-04-23 23:30:32,976 - INFO - ℹ️ Config OpenAI: Status=disabled, Model=gpt-4o, Key Present=True 


In [3]:
class Config(SimpleNamespace):
    ENDPOINT:Path = __root__/'SANDBOX'/'davidka'
    mining_data_path:Path = ENDPOINT/'mining_data'
    crawl_files_list:list = get_filenames_from_directory(mining_data_path, 'json')
    task_description =  Path(ENDPOINT/ 'tasks'/ 'grab_product_page.md').read_text(encoding='utf-8')

driver:SimpleDriver = SimpleDriver()

2025-04-23 23:31:23,221 - WARNING - ⚠️ OpenAI LLM не инициализирован (Key=True, Status=disabled) 
2025-04-23 23:31:23,221 - INFO - ℹ️ Инициализация Gemini: Model=gemini-1.5-flash-latest 
2025-04-23 23:31:23,223 - DEBUG - 🐛 Установлена переменная окружения GOOGLE_API_KEY. 
2025-04-23 23:31:23,225 - INFO - ℹ️ Gemini LLM инициализирован. 
2025-04-23 23:31:23,273 - INFO - ℹ️ Добавлен инструмент WebSearchAPI (SerpApi). 
2025-04-23 23:31:23,273 - WARNING - ⚠️ BrowserController недоступен, браузерные инструменты не добавлены. 
2025-04-23 23:31:23,274 - INFO - ℹ️ Итоговый список инструментов: ['WebSearchAPI'] 


In [4]:
def get_products_urls_list_from_files(crawl_files_list:list = []) -> list:
    """
   Функция читает содержимое файлов  в директории `mining_data`, перемешивает их и возвращает одним большим списком
   """
    products_urls_list = []
    for filename in crawl_files_list or Config.crawl_files_list:
        try:
            file_path = Config.mining_data_path / filename
            crawl_data = j_loads(file_path)['products']
            for product in crawl_data:
                products_urls_list.append(product['product_url'])
        except Exception as ex:
            logger.error(f'Ошибка при обработке файла/n {filename=}/n', ex)
            ...
    random.shuffle(products_urls_list)
    return products_urls_list if isinstance(products_urls_list, list) else [products_urls_list]

def yield_product_urls_from_files(directory: Path = Config.mining_data_path, pattern: str = 'json'):
    """
    Функция возвращает генератор списка `url` Применяется на больших объемах данных
    """
    filenames = get_filenames_from_directory(directory, pattern)
    for filename in filenames:
        try:
            file_path = directory / filename
            crawl_data = j_loads(file_path)['products']
            for product in crawl_data:
                yield product['product_url']
        except Exception as ex:
            logger.error(f'Ошибка при обработке файла {filename=}', ex)
            ...


In [ ]:
# Через генератор для совсем больших данных
# for product_url in yield_product_urls_from_files():

for product_url in get_products_urls_list_from_files():
    try:
        logger.info(f'Обработка URL: {product_url}')
        task = Config.task_description.replace('<URL>', product_url)
        
        extracted_data = await driver.simple_process_task_async(task)
        ipdb.set_trace()
        print(extracted_data)
        ...
    except Exception as ex:
        logger.error(f'Ошибка при обработке {product_url=}', ex)

2025-04-23 23:31:36,956 - INFO - ℹ️ Обработка URL: https://www.unieuro.it/online/Frigoriferi/Marca-Modello-combinato-inox-pidUNI987654321 
2025-04-23 23:31:37,046 - INFO - 🧠 Starting an agent with main_model=models/gemini-1.5-flash-latest +vision, planner_model=None, extraction_model=None 
2025-04-23 23:31:37,066 - INFO - ℹ️ Агент начинает выполнение задачи: "
# Инструкция


**Ты - помощник по обработке страниц товаров.**

## Твое поведение на вебстранице:

### Вначале
1. Перейди на страницу https://www.unieuro.it/online/Frigoriferi/Marca-Modello-combinato-inox-pidUNI987654321 и выполни следующие указания:

2. Если для входа на сайт требуется ввести пароль - прекрати работу 
и верни сообщение:
```markdown
Для сайта https://www.unieuro.it/online/Frigoriferi/Marca-Modello-combinato-inox-pidUNI987654321 
Требуется пароль
```

3. Если есть попап окна (popup) - закрой их перед началом сбора информации

### Далее:

   #### 1. Если это страница товара на которой есть поля товара из словаря - 

> c:\users\user\appdata\local\temp\ipykernel_5016\1231488538.py(11)<module>()



ipdb>  extracted_data


{'history': [AgentHistory(model_output=AgentOutput(current_state=AgentBrain(evaluation_previous_goal='Unknown', memory='I have not performed any actions yet. This is the first step.', next_goal='Go to the specified URL and check for popups or password requirements.'), action=[ActionModel(done=None, search_google=None, go_to_url=GoToUrlAction(url='https://www.unieuro.it/online/Frigoriferi/Marca-Modello-combinato-inox-pidUNI987654321'), go_back=None, wait=None, click_element_by_index=None, input_text=None, save_pdf=None, switch_tab=None, open_tab=None, close_tab=None, extract_content=None, scroll_down=None, scroll_up=None, send_keys=None, scroll_to_text=None, get_dropdown_options=None, select_dropdown_option=None, drag_drop=None)]), result=[ActionResult(is_done=False, success=None, extracted_content='🔗  Navigated to https://www.unieuro.it/online/Frigoriferi/Marca-Modello-combinato-inox-pidUNI987654321', error=None, include_in_memory=True)], state=BrowserStateHistory(url='about:blank', ti

ipdb>  quit


2025-04-23 23:34:24,014 - ERROR - ❌ Ошибка при обработке product_url='https://www.unieuro.it/online/Frigoriferi/Marca-Modello-combinato-inox-pidUNI987654321' 
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_5016\1231488538.py", line 11, in <module>
    print(extracted_data)
    ^^^^^
  File "C:\anaconda3\Lib\bdb.py", line 90, in trace_dispatch
    return self.dispatch_line(frame)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\anaconda3\Lib\bdb.py", line 115, in dispatch_line
    if self.quitting: raise BdbQuit
                      ^^^^^^^^^^^^^
bdb.BdbQuit
2025-04-23 23:34:24,035 - INFO - ℹ️ Обработка URL: https://www.reuters.com/ 
2025-04-23 23:34:24,099 - INFO - 🧠 Starting an agent with main_model=models/gemini-1.5-flash-latest +vision, planner_model=None, extraction_model=None 
2025-04-23 23:34:24,111 - INFO - ℹ️ Агент начинает выполнение задачи: "
# Инструкция


**Ты - помощник по обработке страниц товаров.**

## Твое поведение на вебстранице

> c:\users\user\appdata\local\temp\ipykernel_5016\1231488538.py(11)<module>()

